# Pregunta 3

#### Nombre: Samuel Zúñiga
#### Número de alumno: 16637747

In [3]:
import copy

Primero definimos la clase nodo de nuestro árbol.

In [4]:
class Node(object):
    def __init__(self, value):
        self.value = value
        self.parent = None
        self.left = None
        self.right = None
        self.is_right=None

    def __str__(self):
      return str(self.value)

In [14]:
class MerkleTree :

  def __init__(self , strings, hash_func):

    self.hash_func = hash_func
    
    # seteamos el valor de nuestros nodos hojas
    nodes = list()

    for s in strings:
      n = Node(value=self.hash_func(s))
      nodes.append(n)

    # vemos el caso para cuando tenemos un número impar de nodos en el nivel
    if len(nodes) % 2 != 0:
        nodes.append(copy.deepcopy(nodes[-1]))
    self.leaves = nodes

    # mientras no este en mi nodo raiz
    while len(nodes) != 1: 
      upper_level = list()
      
      # verificamos nuevamente que nuestro nivel sea par
      if len(nodes) % 2 != 0:
        nodes.append(copy.deepcopy(nodes[-1]))

      # recorremos cada 2 nodos para ir asignando valores a los atributos del nodo
      for i in range(len(nodes)//2):
        n1 = nodes[2*i]
        n2 = nodes[2*i+1]

        val = self.hash_func(n1.value + n2.value)

        n_parent = Node(value=val)

        n1.parent = n_parent
        n1.is_right = False

        n2.parent = n_parent
        n2.is_right = True 

        n_parent.left = n1
        n_parent.right = n2

        upper_level.append(n_parent)

      # subimos de nivel en el arbol
      nodes = upper_level

    
    self.root = nodes[0]

  def get_root(self) :
    """
    Returns :
    root : Root of the Merkle Tree
    """
    return self.root


  def get_proof_for (self , item):
    """
    Returns :
    result : None if the item is not part of the leafs of the tree
    A list with the necessary info to prove that the
    item is part of the leafs of the tree
    """

    # creamos la lista de instrucciones a retornar
    instructions = list()

    hashed_item = self.hash_func(item)

    item_node = None

    found = False

    # si tenemos el valor en las hojas podemos seguir haciendo la prueba
    for l in self.leaves:
      if hashed_item == l.value:
        found=True

    # si no tenemos el valor en nuestras hojas retornamos None
    if found == False:
      return None

    
    for n in self.leaves:
      if n.value == hashed_item:
        item_node = n

    #tengo el nodo, debo comenzar a hacer las instrucciones

    current_node = item_node

    while current_node.value != self.root.value:

      # agregamos al nodo hermano y su posición 
      if current_node.is_right:
        instructions.append(tuple([current_node.parent.left.value, "i"]))
      else:
        instructions.append(tuple([current_node.parent.right.value, "d"]))

      current_node = current_node.parent

    return instructions

In [6]:
def verify(root, item, proof, hash_func):

    # hasheamos nuestro item para comenzar a hacer el nodo raíz candidato
    curr = hash_func(item)

    # debido a la construcción del arbol, podemos ir construyendo la raiz dependiendo de la orientación del nodo
    for instruction in proof:
        if instruction[1] == "d":
            curr = hash_func(curr + instruction[0])
        elif instruction[1] == "i":
            curr = hash_func(instruction[0] + curr)

    if curr == root.value:
        return True
    else:
        return False

Algunos tests:

In [15]:
# strings = ["s1","s2","s3","s4","s5","s6"]

In [16]:
# def hash_func(string):
#   return "h("+string+")"

In [17]:
# mt = MerkleTree(strings,hash_func)

In [18]:
# root = mt.get_root()

In [24]:
# print(root)

In [20]:
# proof = mt.get_proof_for("s5")

In [25]:
# print(proof)

In [26]:
# verify(root,"s1",proof,hash_func)

In [27]:
# verify(root,"s5",proof,hash_func)